# CBMPy Tutorial S01 working with SBML Groups

In this section I demonstrate the use of the SBML Level 3 Groups package with CBMPy. Note that, currently, for Groups support the `libsbml-experimental` bindings are required. For more information please see the CBMPy reference guide (available from http://cbmpy.sourceforge.net).

Additional files needed for this tutorial: None.

As always we start by importing CBMPy

In [ ]:
import cbmpy

To begin let's load one of the small built-in models supplied with CBMPy: `cbmpy_test_core` and assign it to the variable `cmod`.

In [ ]:
cmod = cbmpy.readSBML3FBC('cbmpy_test_core')

Let's take a closer look at this model.

In [ ]:
print(cmod.getReactionIds())

In [ ]:
print(cmod.getSpeciesIds())

Let's also look at the reaction annotation (this was stuff classically, and confusingly, stored in the SBML "notes" element)

In [ ]:
print([r.getAnnotations() for r in cmod.reactions])

Note how some reactions have a `subsystem` annotation defined, let's take a look at them and extract the subsystem id's.

In [ ]:
subsystems = []
for R in cmod.reactions:
    print(R.getId(), R.getAnnotation('subsystem'))
    if R.getAnnotation('subsystem') != None and R.getAnnotation('subsystem') not in subsystems:
        subsystems.append(R.getAnnotation('subsystem'))
subsystems.sort()
print('subsystems', subsystems)

We would like to convert these subsystems into and SBML Group. In its most basic form a Group is a, possibly, arbitrary collection of objects which can be collectively treated as a single object for annotation purposes, thus Groups do not effect the evaluation of a model but rather the human interpretation of its elements. Further properties of Groups will be shown below.

Let's begin by simply creating four groups.

In [ ]:
for s in subsystems:
    cmod.createGroup(s)

print('GroupIds', cmod.getGroupIds())

Now that we have our groups we can add reaction objects. Note that in CBMPy the object itself is added to the Group. We begin by adding R23 and R24 to C4 manually.

In [ ]:
C4 = cmod.getGroup('C4')

In [ ]:
C4.addMember(cmod.getReaction('R23'))
C4.addMember(cmod.getReaction('R24'))
print('C4 member ids', C4.getMemberIDs())

Of course this can easily be scripted so we will delete the C4 members and do everything at once.

In [ ]:
C4.deleteMember('R23')
C4.deleteMember('R24')
print('C4 member ids', C4.getMemberIDs())

In [ ]:
for R in cmod.reactions:
    if R.getAnnotation('subsystem') != None:
        print(R.getId(), R.getAnnotation('subsystem'))
        cmod.getGroup(R.getAnnotation('subsystem')).addMember(R)
        R.deleteAnnotation('subsystem')
        

In [ ]:
for G in cmod.groups:
    print(G.getId(), G.getMemberIDs())

Excellent. We now have have groups of reactions but how can we label them as being a subsystem, there are various ways, we could use the Group `name` or `notes` field, CBMPy annotation or even use the (systems biology ontology) SBO or GO term to describe the pathway. Let's try a combination of these.

In [ ]:
for G in cmod.groups:
    G.setName(G.getId()) # sets the group name to its initial ID
    G.setNotes('subsystem')
    G.setAnnotation('subsystem', True)
    G.setSBOterm('SBO:0000000') # as an example, this is not a meaningful SBO term


What we have been doing here is labelling the group itself, not the individual members of the group (no inheritance). If we would actually like the members of the group to be labelled/annotated using the Group as a superclass then we need to set the member annotation directly. Again using the C4 Group we can either set the `shared` properties directly:

In [ ]:
print(C4.getMemberIDs())
C4.setSharedNotes('subsystem')

In [ ]:
print('C4 shared', C4.getSharedNotes())
print('R23 notes', cmod.getReaction('R23').getNotes())
print('R24 notes', cmod.getReaction('R24').getNotes())

Currently the new `notes` annotation is an annotation to the group's list of members. This is the normal Groups behaviour, in addition, CBMPy provides functionality that allows one to push the annotation to the group's member objects. In this case not overwriting any existing `notes` (if they exist), only adding them to members where the attribute is undefined (set with the flag `overwrite=False`).

In [ ]:
C4.assignSharedNotesToMembers(overwrite=False)
print('C4 shared', C4.getSharedNotes())
print('R23 notes', cmod.getReaction('R23').getNotes())
print('R24 notes', cmod.getReaction('R24').getNotes())

Similarly annotations, MIRIAM annotations and SBO terms can be pushed to members, see the group *assignX* methods for more details. Now we can try add an SBO term to the C1 group *without* pushing it to the members.

In [ ]:
cmod.getGroup('C1').setSharedSBOterm('SBO:0000000') # SBO:0000000 is an example only

Groups can be defined as either a "collection", "classification" or "partonomy" by default groups are classified as an (arbitrary) collection:

In [ ]:
cmod.getGroup('C2').setKind('partonomy')
cmod.getGroup('C3').setKind('classification')

print('C1 kind:', cmod.getGroup('C1').getKind())
print('C2 kind:', cmod.getGroup('C2').getKind())
print('C3 kind:', cmod.getGroup('C3').getKind())

Finally, we write the file to an SBML Level 3 FBC version 1 file.

In [ ]:
cbmpy.writeSBML3FBC(cmod, 'groups_example.xml', add_groups=True)

This basic introduction of the CBMPy Groups functionality should allow you to create custom groups of objects (including Groups). Additional group functionality is conntinuously being implemented, please see the reference guide and the Group object docstrings and methods for more details. For a final test let's load our saved file that includes the groups data.

In [ ]:
dmod = cbmpy.readSBML3FBC('groups_example.xml')

In [ ]:
print('Group IDs', dmod.getGroupIds())